In [1]:
# determine the optimal pre-match forecast, given elo_diff and s_elo_diff
# step 3: calculate glicko and s_glicko for each player and do the same

In [8]:
from sklearn import linear_model
from sklearn.model_selection import KFold
from sklearn.metrics import log_loss,accuracy_score

# cols is a list of all column sets to test; compare with kls pre-match forecasts
def validate_results(df,columns,n_splits=5):
    kfold = KFold(n_splits=n_splits,shuffle=True)
    scores = np.zeros([len(columns)+2,2,n_splits]);i=0
    for train_ind,test_ind in kfold.split(df):
        lm = linear_model.LogisticRegression(fit_intercept = True)
        train_df,test_df = df.loc[train_ind],df.loc[test_ind]
        
        for k,cols in enumerate(columns):
            lm.fit(train_df[cols].values.reshape([len(train_df),len(cols)]),train_df['winner'])
            y_preds = lm.predict(test_df[cols].values.reshape([len(test_df),len(cols)]))
            y_probs = lm.predict_proba(test_df[cols].values.reshape([len(test_df),len(cols)]))
            scores[k][0][i]=accuracy_score(test_df['winner'],y_preds)
            scores[k][1][i]=log_loss(test_df['winner'],y_probs,labels=[0,1])
        
        y_preds2 = test_df['match_prob_kls']>.5
        y_preds3 = test_df['match_prob_kls_JS']>.5
        scores[len(columns)][0][i]=accuracy_score(test_df['winner'],y_preds2)
        scores[len(columns)][1][i]=log_loss(test_df['winner'],test_df['match_prob_kls'],labels=[0,1])
        scores[len(columns)+1][0][i]=accuracy_score(test_df['winner'],y_preds3)
        scores[len(columns)+1][1][i]=log_loss(test_df['winner'],test_df['match_prob_kls_JS'],labels=[0,1])
        i+=1
    
    for i,cols in enumerate(columns):
        print 'columns: ',cols
        #print '% s_elo used in lm fit: ',lm.coef_[0][1]/(lm.coef_[0][0]+lm.coef_[0][1])
        print 'accuracy: ', np.mean(scores[i][0])
        print 'loss: ', np.mean(scores[i][1])
    
    print 'kls probabilities'
    print 'accuracy: ', np.mean(scores[len(columns)][0])
    print 'loss: ', np.mean(scores[len(columns)][1])

    print 'kls JS probabilities'
    print 'accuracy: ', np.mean(scores[len(columns)+1][0])
    print 'loss: ', np.mean(scores[len(columns)+1][1])

In [2]:
import sys
sys.path.insert(0,'/Users/jacobgollub/Desktop/college (current)/research/pbp_explorations/scripts/sackmann')
import tennisGameProbability,tennisMatchProbability,tennisSetProbability,tennisTiebreakProbability
from tennisMatchProbability import matchProb
import pandas as pd
import numpy as np

# can test this on our subset of 10,000 matches as well as all matches in the database:
#df = pd.read_csv('../my_data/elo_pbp_with_surface_8_27.csv')
df = pd.read_csv('../my_data/elo_atp_matches_21st_century.csv')
del df['Unnamed: 0']

# currently looking at 2014 tour-level matches, excluding Davis Cup
df = df[df['match_year']==2014].reset_index(drop=True)
df = df[df['tourney_name']!='Davis Cup'].reset_index(drop=True)
df.head(2)

,tourney_id,tourney_name,tourney_date,match_year,p0_name,p1_name,p0_elo,p1_elo,elo_diff,p0_s_elo,...,tourney_stats,best_of,score,pbp,winner,match_id,p0_s_kls,p1_s_kls,p0_s_kls_JS,p1_s_kls_JS
0,2014-301,Auckland,2014-01-06,2014,Donald Young,Federico Delbonis,1546.540528,1711.117324,-164.576796,1529.776774,...,0.658301,3,7-5 6-2,SSSRS;SSRSRA;RSRRSSSRSS;SRSSS;SSSS;SRRSSS;RSSR...,1,42810,0.572116,0.552431,0.591849,0.580801
1,2014-301,Auckland,2014-01-06,2014,Albert Montanes,Santiago Giraldo,1711.293427,1645.679649,65.613778,1512.891795,...,0.658301,3,6-1 7-6,RRARSR;SRSDSS;RRSSRSRSSS;RSSAS;SRSRDSRSRSRR;SR...,0,42811,0.624996,0.615394,0.625741,0.616902


In [3]:
# generate match probabilities and z-scores for Klaassen method, with and w/o JS estimators
df['match_prob_kls'] = [matchProb(row['p0_s_kls'],1-row['p1_s_kls']) for i,row in df.iterrows()]
df['match_prob_kls_JS'] = [matchProb(row['p0_s_kls_JS'],1-row['p1_s_kls_JS']) for i,row in df.iterrows()]

from scipy.stats import norm
for col in ['match_prob_kls','match_prob_kls_JS']:
    df[col.replace('prob','z')] = norm.ppf(df[col])
print df.shape

(2409, 42)


In [4]:
print 'elo baseline: ',  sum((df['elo_diff']>0) == df['winner'])/float(len(df))
print 'surface elo baseline: ', sum((df['s_elo_diff']>0) == df['winner'])/float(len(df))

current = (0,0)
for weight in np.arange(10)*.1:
    accuracy = sum(((1-weight)*df['elo_diff']+weight*df['s_elo_diff']>0) == df['winner'])/float(len(df))
    if accuracy > current[1]:
        current=weight,accuracy
print 'best s_elo weight: ',current[0],'accuracy =',current[1]
#df['elo_diff_weighted'] = (1-current[1])*df['elo_diff']+current[1]*df['s_elo_diff']

elo baseline:  0.69198837692
surface elo baseline:  0.686176836862
best s_elo weight:  0.3 accuracy = 0.695724366957


In [ ]:
# seems worth trying methods to alter match_prob_kls with elo

In [15]:
cols = [['elo_diff','s_elo_diff'],['elo_diff','s_elo_diff','match_z_kls'],\
        ['elo_diff','s_elo_diff','match_z_kls_JS']]
n_splits = 5
validate_results(df,cols,n_splits)

columns:  ['elo_diff', 's_elo_diff']
accuracy:  0.688668144685
loss:  0.580059974458
columns:  ['elo_diff', 's_elo_diff', 'match_z_kls']
accuracy:  0.720630429344
loss:  0.527242740716
columns:  ['elo_diff', 's_elo_diff', 'match_z_kls_JS']
accuracy:  0.718141665445
loss:  0.538486150702
kls probabilities
accuracy:  0.7256088198
loss:  0.531296893563
kls JS probabilities
accuracy:  0.721461167519
loss:  0.545266605368


In [6]:
# now, create win_prob columns for elo

In [9]:
# logistic regression on z scores of our probabilities AND elo_diff/s_elo_diff
from scipy.stats import norm
df_all = pd.read_csv('../my_data/feature_df_pbp3_8_23_alphas.csv')
del df_all['Unnamed: 0']
for col in ['sackmann_prob','klaassen_prob','klaassen_prob_a_400']:
    df_all[col.replace('prob','z')] = norm.ppf(df_all[col])
df_all.loc[df_all['sackmann_z']>10,'sackmann_z'] = 10
df_all.loc[df_all['klaassen_z']>10,'klaassen_z'] = 10
df_all.loc[df_all['klaassen_z_a_400']>10,'klaassen_z_a_400'] = 10

In [11]:
from sklearn import linear_model
from sklearn.model_selection import KFold
from sklearn.metrics import log_loss,accuracy_score

# also will want to try ensemble methods at some point...
kfold = KFold(n_splits=5,shuffle=True)
scores = [],[]
cols = ['elo_diff','s_elo_diff','klaassen_z_a_400']
for train_ind,test_ind in kfold.split(df_all):
    lm = linear_model.LogisticRegression(fit_intercept = True)
    train_df,test_df = df_all.loc[train_ind],df_all.loc[test_ind]
    lm.fit(train_df[cols].values.reshape([len(train_df),len(cols)]),train_df['winner'])
    y_preds = lm.predict(test_df[cols].values.reshape([len(test_df),len(cols)]))
    y_probs = lm.predict_proba(test_df[cols].values.reshape([len(test_df),len(cols)]))
    accuracy = accuracy_score(test_df['winner'],y_preds)
    loss = log_loss(test_df['winner'],y_probs,labels=[0,1])
    scores[0].append(accuracy);scores[1].append(loss)
#print '% s_elo used in lm fit: ',lm.coef_[0][1]/(lm.coef_[0][0]+lm.coef_[0][1])
print 'coefficients: ', lm.coef_[0]
print 'accuracy: ', np.mean(scores[0])
print 'loss: ', np.mean(scores[1])

coefficients:  [ -5.39480879e-04   1.72440109e-03   1.71413858e+00]
accuracy:  0.778048803942
loss:  0.456072287662


In [ ]:
# perform same analysis across entire match dataset, over the decades

In [161]:
# one-time fix to transfer over the elo_diff's
# import pandas as pd
# import numpy as np

# df_og = pd.read_csv('../my_data/elo_pbp_with_surface_8_22.csv')
# del df_og['Unnamed: 0']
# df = pd.read_csv('../my_data/feature_df_pbp3_8_23_alphas.csv')
# del df['Unnamed: 0']

# elo_dict = dict(zip(range(len(df_og)),df_og['elo_diff']))
# s_elo_dict = dict(zip(range(len(df_og)),df_og['s_elo_diff']))

# df['elo_diff'] = [elo_dict[m_id] for m_id in df['match_id']]
# df['s_elo_diff'] = [s_elo_dict[m_id] for m_id in df['match_id']]
# df.to_csv('../my_data/feature_df_pbp3_8_23_alphas.csv')

In [ ]:
# from sklearn import linear_model
# from sklearn.model_selection import KFold
# from sklearn.metrics import log_loss,accuracy_score

# n_splits = 5
# kfold = KFold(n_splits=n_splits,shuffle=True)
# scores = np.zeros([3,2,n_splits]);i=0
# cols = ['elo_diff','s_elo_diff']
# for train_ind,test_ind in kfold.split(df):
#     lm = linear_model.LogisticRegression(fit_intercept = True)
#     train_df,test_df = df.loc[train_ind],df.loc[test_ind]
#     lm.fit(train_df[cols].values.reshape([len(train_df),len(cols)]),train_df['winner'])
#     y_preds = lm.predict(test_df[cols].values.reshape([len(test_df),len(cols)]))
#     y_preds2 = test_df['match_prob_kls']>.5
#     y_preds3 = test_df['match_prob_kls_JS']>.5
#     y_probs = lm.predict_proba(test_df[cols].values.reshape([len(test_df),len(cols)]))
#     scores[0][0][i]=accuracy_score(test_df['winner'],y_preds)
#     scores[0][1][i]=log_loss(test_df['winner'],y_probs,labels=[0,1])
#     scores[1][0][i]=accuracy_score(test_df['winner'],y_preds2)
#     scores[1][1][i]=log_loss(test_df['winner'],test_df['match_prob_kls'],labels=[0,1])
#     scores[2][0][i]=accuracy_score(test_df['winner'],y_preds3)
#     scores[2][1][i]=log_loss(test_df['winner'],test_df['match_prob_kls_JS'],labels=[0,1])

    
#     i+=1
# print '% s_elo used in lm fit: ',lm.coef_[0][1]/(lm.coef_[0][0]+lm.coef_[0][1])
# print 'accuracy: ', np.mean(scores[0][0])
# print 'loss: ', np.mean(scores[0][1])

# print 'kls probabilities'
# print 'accuracy: ', np.mean(scores[1][0])
# print 'loss: ', np.mean(scores[1][1])

# print 'kls JS probabilities'
# print 'accuracy: ', np.mean(scores[2][0])
# print 'loss: ', np.mean(scores[2][1])